# 99 Scala Exercises (20 to 28)

### 20\. Remove the Kth element from a list.

In [4]:
def removeAt[A](n: Int, l: List[A]): Option[(List[A], A)] = l splitAt(n) match {
    case (xs, y::ys) => Some((xs ++ ys) -> y)
    case _ => None
}
removeAt(1, List('a, 'b, 'c, 'd))

defined function removeAt
res3_1: Option[(List[Symbol], Symbol)] = Some((List('a, 'c, 'd), 'b))

###  21\. Insert an element at a given position into a list.

In [6]:
def insertAt[A](e: A, n: Int, l: List[A]): Option[List[A]] = l splitAt(n) match {
    case (xs, ys) => Some(xs ::: e :: ys)
    case _ => None
}
insertAt('new, 1, List('a, 'b, 'c, 'd))

defined function insertAt
res5_1: Option[List[Symbol]] = Some(List('a, 'new, 'b, 'c, 'd))

### 22\. Create a list containing all integers within a given range.

In [13]:
// Built-in
def range(x: Int, y: Int): List[Int] = (x to y).toList

// Tail recursive
def rangeTailRec(from: Int, to: Int): List[Int] = {
    @annotation.tailrec
    def inner(i: Int, accum: List[Int]): List[Int] =
        if (i <= to) inner(i+1, i +: accum) else accum.reverse 
    inner(from, List.empty)
}

range(4, 9)
rangeTailRec(4, 9)

time(range(1, 10000))._2
time(rangeTailRec(1, 10000))._2

defined function range
defined function rangeTailRec
res12_2: List[Int] = List(4, 5, 6, 7, 8, 9)
res12_3: List[Int] = List(4, 5, 6, 7, 8, 9)
res12_4: concurrent.duration.FiniteDuration = 208199 nanoseconds
res12_5: concurrent.duration.FiniteDuration = 2160717 nanoseconds

### 23\. Extract a given number of randomly selected elements from a list.

In [35]:
// Tail recursive
def randomSelect[A](n: Int, l: List[A]): List[A] = {
    @annotation.tailrec
    def inner(i: Int, innerList: List[A], accum: List[A]): List[A] = innerList match {
        case Nil | _ if i >= n => accum
        case xs => {
            val index = scala.util.Random.nextInt(xs.length)
            val (la, a) = removeAt(index, xs).get
            inner(i+1, la, a +: accum)
        }
    }
    inner(0, l, List.empty)
}

randomSelect(3, List('a, 'b, 'c, 'd, 'f, 'g, 'h))

defined function randomSelect
res34_1: List[Symbol] = List('a, 'd, 'c)

### 24\. Lotto: Draw N different random numbers from the set 1..M.

In [38]:
def lotto(n: Int, m: Int) = randomSelect(n, List.range(1, m))
lotto(6, 49)

defined function lotto
res37_1: List[Int] = List(42, 48, 5, 6, 36, 47)

### 25\. Generate a random permutation of the elements of a list.

In [37]:
def randomPermute[A](l: List[A]) = randomSelect(l.length, l)
randomPermute(List('a, 'b, 'c, 'd, 'e, 'f))

defined function randomPermute
res36_1: List[Symbol] = List('a, 'f, 'd, 'c, 'b, 'e)

### 26\. Generate the combinations of K distinct objects chosen from the N elements of a list.
In how many ways can a committee of 3 be chosen from a group of 12 people? We all know that there are C(12,3) = 220 possibilities (C(N,K) denotes the well-known binomial coefficient). For pure mathematicians, this result may be great. But we want to really generate all the possibilities.

In [1]:
// Built-in
def combinations[A](k: Int, la: List[A]): List[List[A]] = la.combinations(k).toList

// Ported from Haskell Solution
def combinationsHaskell[A](k: Int, la: List[A]): List[List[A]] = (k, la) match {
    case (0, _)     => List(Nil)
    case (_, Nil)   => Nil 
    case (_, x::xs) => (combinations(k-1, xs) map (x :: _)) ::: combinations(k, xs) 
}

combinations(3, List('a, 'b, 'c, 'd, 'e, 'f))
combinationsHaskell(3, List('a, 'b, 'c, 'd, 'e, 'f))

time(combinations(3, (1 to 100).toList))._2
time(combinationsHaskell(3, (1 to 100).toList))._2

defined function combinations
defined function combinationsHaskell
res0_2: List[List[Symbol]] = List(
  List('a, 'b, 'c),
  List('a, 'b, 'd),
  List('a, 'b, 'e),
  List('a, 'b, 'f),
  List('a, 'c, 'd),
  List('a, 'c, 'e),
  List('a, 'c, 'f),
  List('a, 'd, 'e),
  List('a, 'd, 'f),
  List('a, 'e, 'f),
  List('b, 'c, 'd),
...
res0_3: List[List[Symbol]] = List(
  List('a, 'b, 'c),
  List('a, 'b, 'd),
  List('a, 'b, 'e),
  List('a, 'b, 'f),
  List('a, 'c, 'd),
  List('a, 'c, 'e),
  List('a, 'c, 'f),
  List('a, 'd, 'e),
  List('a, 'd, 'f),
  List('a, 'e, 'f),
  List('b, 'c, 'd),
...
res0_4: concurrent.duration.FiniteDuration = 948238441 nanoseconds
res0_5: concurrent.duration.FiniteDuration = 266373003 nanoseconds

### 27\. Group the elements of a set into disjoint subsets.
a) In how many ways can a group of 9 people work in 3 disjoint subgroups of 2, 3 and 4 persons? Write a function that generates all the possibilities.

In [17]:
type Groups[A] = List[List[List[A]]] 

def group3[A](la: List[A]): Groups[A] =  for {
    g2 <- combinations(2, la)
    g3 <- combinations(3, la diff g2)
    g4 = la diff(g2 ++ g3)
} yield List(g2, g3, g4)

def group[A](lk: List[Int], la: List[A]): Groups[A] = lk match {
    case x::Nil => List(combinations(x, la))
    case x::xs  => combinations(x, la) flatMap { c => group(xs, la diff c).map(c::_) }
    case _ => List(Nil)
}

val l1 = group3(List("Aldo", "Beat", "Carla", "David", "Evi", "Flip", "Gary", "Hugo", "Ida"))
val l2 = group(List(2, 3, 4), List("Aldo", "Beat", "Carla", "David", "Evi", "Flip", "Gary", "Hugo", "Ida"))
l1 == l2

defined type Groups
defined function group3
defined function group
l1: wrapper.wrapper.Groups[String] = List(List(List(Aldo, Beat), List(Carla, David, Evi), List(Flip, Gary, Hugo, Ida)), List(List(Aldo, Beat), List(Carla, David, Flip), List(Evi, Gary, Hugo, Ida)), List(List(Aldo, Beat), List(Carla, David, Gary), List(Evi, Flip, Hugo, Ida)), List(List(Aldo, Beat), List(Carla, David, Hugo), List(Evi, Flip, Gary, Ida)), List(List(Aldo, Beat), List(Carla, David, Ida), List(Evi, Flip, Gary, Hugo)), List(List(Aldo, Beat), List(Carla, Evi, Flip), List(David, Gary, Hugo, Ida)), List(List(Aldo, Beat), List(Carla, Evi, Gary), List(David, Flip, Hugo, Ida)), List(List(Aldo, Beat), List(Carla, Evi, Hugo), List(David, Flip, Gary, Ida)), List(List(Aldo, Beat), List(Carla, Evi, Ida), List(David, Flip, Gary, Hugo)), List(List(Aldo, Beat), List(Carla, Flip, Gary), List(David, Evi, Hugo, Ida)), List(List(Aldo, Beat), List(Carla, Flip, Hugo), List(David, Evi, Gary, Ida)), List(List(Aldo, Beat), List(Carla

### 28\. Sorting a list of lists according to length of sublists.
a) We suppose that a list contains elements that are lists themselves. The objective is to sort the elements of the list according to their length. E.g. short lists first, longer lists later, or vice versa.

In [57]:
// Using built-in functions
def lsort[A](lla: List[List[A]]): List[List[A]] =
    lla sortWith(_.length > _.length) 

lsort(List(List('a, 'b, 'c),
           List('d, 'e),
           List('f, 'g, 'h),
           List('d, 'e),
           List('i, 'j, 'k, 'l),
           List('m, 'n),
           List('o)))

defined function lsort
res56_1: List[List[Symbol]] = List(
  List('i, 'j, 'k, 'l),
  List('a, 'b, 'c),
  List('f, 'g, 'h),
  List('d, 'e),
  List('d, 'e),
  List('m, 'n),
  List('o)
)

b) Again, we suppose that a list contains elements that are lists themselves. But this time the objective is to sort the elements according to their length frequency; i.e. in the default, sorting is done ascendingly, lists with rare lengths are placed, others with a more frequent length come later.

In [56]:
def lsortFreq[A](lla: List[List[A]]): List[List[A]] = {
    val zippedWithLength = lla.map (l => l -> l.length)
    val zippedWithFreq = zippedWithLength map {
        case (xs, len) => xs -> zippedWithLength.count { _._2 == len}
    }
   zippedWithFreq.sortBy(_._2).map(_._1)
}

// Uses List.view
def lsortFreqLazy[A](lla: List[List[A]]): List[List[A]] = {
    val zippedWithLength = lla.view.map (l => l -> l.length)
    val zippedWithFreq = zippedWithLength map {
        case (xs, len) => xs -> zippedWithLength.count { _._2 == len}
    }
   zippedWithFreq.sortBy(_._2).map(_._1).force.toList
}

val l = List(List('a, 'b, 'c),
         List('d, 'e),
         List('f, 'g, 'h),
         List('d, 'e),
         List('i, 'j, 'k, 'l),
         List('m, 'n),
         List('o))

lsortFreq(l)
lsortFreqLazy(l)

time(lsortFreq(l))._2
time(lsortFreqLazy(l))._2

defined function lsortFreq
defined function lsortFreqLazy
l: List[List[Symbol]] = List(
  List('a, 'b, 'c),
  List('d, 'e),
  List('f, 'g, 'h),
  List('d, 'e),
  List('i, 'j, 'k, 'l),
  List('m, 'n),
  List('o)
)
res55_3: List[List[Symbol]] = List(
  List('i, 'j, 'k, 'l),
  List('o),
  List('a, 'b, 'c),
  List('f, 'g, 'h),
  List('d, 'e),
  List('d, 'e),
  List('m, 'n)
)
res55_4: List[List[Symbol]] = List(
  List('i, 'j, 'k, 'l),
  List('o),
  List('a, 'b, 'c),
  List('f, 'g, 'h),
  List('d, 'e),
  List('d, 'e),
  List('m, 'n)
)
res55_5: concurrent.duration.FiniteDuration = 71724 nanoseconds
res55_6: concurrent.duration.FiniteDuration = 181695 nanoseconds